In [7]:
import os
import sqlite3

import numpy as np
import pandas as pd
from datasets import load_dataset, load_from_disk

In [12]:
df = pd.read_csv("../preprocessed/331_retrieve_b/b_bge_10_4_3/data2.csv")
preds = np.load("../output/350_1st_infer/b_bge_10_4_3/data2_pred.npy")


option_to_index = {option: idx for idx, option in enumerate("ABCDE")}
index_to_option = {v: k for k, v in option_to_index.items()}


def predictions_to_map_output_top(predictions):
    sorted_answer_indices = np.argsort(-predictions)  # Sortting indices in descending order
    top_answer_indices = sorted_answer_indices[:, :]  # Taking the first three indices for each row
    top_answers = np.vectorize(index_to_option.get)(
        top_answer_indices
    )  # Transforming indices to options - i.e., 0 --> A
    return np.apply_along_axis(lambda row: " ".join(row[0]), 1, top_answers)


valid_preds = predictions_to_map_output_top(preds[: len(df), :])
df["preds"] = valid_preds

df[df["preds"] != df["answer"]].index

Int64Index([  32,   39,   65,   68,   96,  120,  124,  135,  147,  178,
            ...
            1148, 1157, 1159, 1165, 1180, 1182, 1183, 1185, 1188, 1194],
           dtype='int64', length=202)

In [2]:
paraphs_parsed_dataset = load_from_disk("../input/all-paraphs-parsed-expanded")
modified_texts = paraphs_parsed_dataset.map(
    lambda example: {
        "temp_text": f"{example['title']} {example['section']} {example['text']}".replace("\n", " ").replace("'", "")
    },
    num_proc=2,
)["temp_text"]

wiki_df = pd.DataFrame({"text": modified_texts})

Map (num_proc=2):   0%|          | 0/2101279 [00:00<?, ? examples/s]

In [4]:
modified_texts[0]

'Russell Epstein Russell Epstein Russell Epstein is a professor of psychology at the University of Pennsylvania, who studies neural mechanisms underlying visual scene perception, event perception, object recognition, and spatial navigation in humans. His lab studies the role of the Parahippocampal and retrosplenial cortices in determining how people orient themselves relative to their surroundings.'

In [15]:
%%time
file_path = ":memory:"

if os.path.exists(file_path):
    # ファイルを削除
    os.remove(file_path)
db = sqlite3.connect(file_path)
cur = db.cursor()
cur.execute('create virtual table imdb using fts5(text, tokenize="porter unicode61");')  # UNINDEXED

CPU times: user 2.3 ms, sys: 254 µs, total: 2.55 ms
Wall time: 1.23 ms


In [18]:
%%time
cur.executemany(
    "insert into imdb (text) values (?);",
    wiki_df[["text"]].to_records(index=False),
)
db.commit()

CPU times: user 1min 57s, sys: 1.32 s, total: 1min 58s
Wall time: 1min 58s


In [19]:
index = 0
text = (
    df.iloc[index].prompt
    + " "
    + df.iloc[index].A
    + " "
    + df.iloc[index].B
    + " "
    + df.iloc[index].C
    + " "
    + df.iloc[index].D
    + " "
    + df.iloc[index].E
)
text = re.sub(r"[^a-zA-Z0-9]", " ", text)
filtered_words = [word for word in text.split() if word.lower() not in stop_words]

In [21]:
%%time

q = " OR ".join(filtered_words)

res = cur.execute(
    f"""select text, rank
                      from imdb
                      where text MATCH "{q}"
                      ORDER BY rank
                      limit 1""",
).fetchall()

print(text)
print()
print(q)

res

Which of the following statements accurately describes the impact of Modified Newtonian Dynamics  MOND  on the observed  missing baryonic mass  discrepancy in galaxy clusters  MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called  fuzzy dark matter   MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20  MOND is a theory that explains the missing baryonic mass in galaxy clusters that was previously considered dark matter by demonstrating that the mass is in the form of neutrinos and axions  MOND is a theory that reduces the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 2  MOND is a theory that eliminates the observed missing baryonic mass in g

[('Modified Newtonian dynamics Modified Newtonian dynamics MOND is an example of a class of theories known as modified gravity, and is an alternative to the hypothesis that the dynamics of galaxies are determined by massive, invisible dark matter halos. Since Milgroms original proposal, proponents of MOND have claimed to successfully predict a variety of galactic phenomena that they state are difficult to understand as consequences of dark matter.Though MOND explains the anomalously great rotational velocities of galaxies at their perimeters, it does not fully explain the velocity dispersions of individual galaxies within galaxy clusters. MOND reduces the discrepancy between the velocity dispersions and clusters observed missing baryonic mass from a factor of around 10 to a factor of about 2. However, the residual discrepancy cannot be accounted for by MOND, requiring that other explanations close the gap such as the presence of as-yet undetected missing baryonic matter.The accurate me

Int64Index([  32,   39,   65,   68,   96,  120,  124,  135,  147,  178,
            ...
            1148, 1157, 1159, 1165, 1180, 1182, 1183, 1185, 1188, 1194],
           dtype='int64', length=202)

In [7]:
index = 1148
print(preds[index])
df.iloc[index]

[0.63688767 0.05020387 0.2634867  0.01801323 0.03140848]


prompt            What structures form the root of the penis?
A           The diverging corpora cavernosa, covered by th...
B           The diverging corpora cavernosa, covered by th...
C           The diverging crura, covered by the Ischiocave...
D           The diverging crura, covered by the Bulbospong...
E           The diverging corpora cavernosa, covered by th...
answer                                                      C
context     Root of penis > Each crus is covered by the is...
sim_max                                               0.19204
sim_min                                              0.108913
sim_mean                                             0.174435
sim_std                                              0.022915
sim_num                                                    12
preds                                                       A
Name: 1148, dtype: object

In [8]:
print(df.iloc[index].prompt)
print()
print(df.iloc[index][df.iloc[index].answer])
print()
print(df.iloc[index].context)

What structures form the root of the penis?

The diverging crura, covered by the Ischiocavernosus muscle, and the median urethral bulb surrounded by the Bulbospongiosus.

Root of penis > Each crus is covered by the ischiocavernosus muscle which helps maintain an erection by contracting to force blood from the crura into the corpora cavernosa. The bulb is surrounded by the bulbospongiosus muscle which contracts to help empty the urethra of any residual semen and urine. It also helps maintain the erection by increasing the pressure in the bulb. During orgasm, the muscles surrounding the root contract involuntarily pushing the semen towards the urinary meatus. Root of penis > During arousal, the root and the corpora cavernosa engorge with blood and become rigid (erection). Meanwhile, the corpus spongiosum remains pliable making the urethra a viable channel for semen during ejaculation. There are four muscles associated with the root of the penis; a pair of ischiocavernosus and a pair of b

In [50]:
df = pd.read_parquet("../input/llm-science-wikipedia-data-b/b.parquet")
df

,id,title,text
0,59737009,Biodiversity Monitoring Switzerland,The Biodiversity Monitoring Switzerland (BDM) ...
1,59737081,Bigoudène,"In Breton tradition, a coiffe bigoudène is a w..."
2,59737128,Barnabas Bala,Barnabas Yusuf Bala (20 December 1956 – 11 Jul...
3,59739888,Bagdah,Bagdah or Bagdaha may refer to: Bagdah (commun...
4,59739916,Big John Hamilton,Big John Hamilton may refer to: Big John Hamil...
...,...,...,...
351878,30045543,Bill Reeder,"William Edgar Reeder (February 20, 1922 – Marc..."
351879,3004563,"Black River Public School (Holland, Michigan)",Black River Public School is a public college ...
351880,30045661,Breg pri Konjicah,Breg pri Konjicah (pronounced [ˈbɾeːk pɾi kɔˈn...
351881,30045729,Brezje pri Ločah,Brezje pri Ločah (pronounced [ˈbreːzjɛ pɾi ˈlo...


In [52]:
window_size = 4
sliding_size = 3
filter_len = 3
filter_len_max = 1000
import blingfire as bf
from pandarallel import pandarallel
from tqdm.auto import tqdm

pandarallel.initialize(progress_bar=True)


def extract_chunk_by_sliding_window(text_list: list[str], window_size: int, sliding_size: int) -> list[str]:
    """
    text のリストをsliding windowで結合する。window_size個のtextが含まれるまで結合し、sliding_size個ずつずらして結合する。
    """
    chunks = []
    for i in range(0, len(text_list), sliding_size):
        chunk = " ".join(text_list[i : i + window_size])
        chunks.append(chunk)
    return chunks


def split_sentences(text):
    document = text.replace("\n", " ")
    _, sentence_offsets = bf.text_to_sentences_and_offsets(document)
    section_sentences = []
    for o in sentence_offsets:
        if filter_len < o[1] - o[0] and o[1] - o[0] < filter_len_max:
            section_sentences.append(document[o[0] : o[1]])
    chunks = extract_chunk_by_sliding_window(section_sentences, window_size, sliding_size)
    return chunks


df["text"] = df["text"].parallel_apply(split_sentences)
df = df.explode(["text"]).reset_index(drop=True)
df["text"] = df["title"] + " > " + df["text"]
df

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


,id,title,text
0,59737009,Biodiversity Monitoring Switzerland,Biodiversity Monitoring Switzerland > The Biod...
1,59737009,Biodiversity Monitoring Switzerland,Biodiversity Monitoring Switzerland > Biodiver...
2,59737009,Biodiversity Monitoring Switzerland,Biodiversity Monitoring Switzerland > Together...
3,59737009,Biodiversity Monitoring Switzerland,Biodiversity Monitoring Switzerland > The Biod...
4,59737009,Biodiversity Monitoring Switzerland,Biodiversity Monitoring Switzerland > Data on ...
...,...,...,...
5380129,30045865,"Board of Intermediate and Secondary Education,...","Board of Intermediate and Secondary Education,..."
5380130,30045865,"Board of Intermediate and Secondary Education,...","Board of Intermediate and Secondary Education,..."
5380131,30045865,"Board of Intermediate and Secondary Education,...","Board of Intermediate and Secondary Education,..."
5380132,30045865,"Board of Intermediate and Secondary Education,...","Board of Intermediate and Secondary Education,..."


In [36]:
"""
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer("english")  # 使用言語に応じて変更
stop_words = set(stopwords.words("english"))  # 使用言語に応じて変更


def clean_text_for_fts(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    text = " ".join([i.strip().lower() for i in text.split()])
    filtered_words = [stemmer.stem(word) for word in text.split() if word not in stop_words]
    return text


df["clean_title"] = df["title"].apply(clean_text_for_fts)
df["clean_text"] = df["text"].apply(clean_text_for_fts)
df
"""

'\nimport re\n\nimport nltk\nfrom nltk.corpus import stopwords\nfrom nltk.stem import SnowballStemmer\n\nstemmer = SnowballStemmer("english")  # 使用言語に応じて変更\nstop_words = set(stopwords.words("english"))  # 使用言語に応じて変更\n\n\ndef clean_text_for_fts(text):\n    text = re.sub(r"[^a-zA-Z0-9]", " ", text)\n    text = " ".join([i.strip().lower() for i in text.split()])\n    filtered_words = [stemmer.stem(word) for word in text.split() if word not in stop_words]\n    return text\n\n\ndf["clean_title"] = df["title"].apply(clean_text_for_fts)\ndf["clean_text"] = df["text"].apply(clean_text_for_fts)\ndf\n'

In [5]:
%%time
import os

file_path = "tutorial.db"

if os.path.exists(file_path):
    # ファイルを削除
    os.remove(file_path)
db = sqlite3.connect(file_path)
cur = db.cursor()
cur.execute('create virtual table imdb using fts5(text , title UNINDEXED, tokenize="porter unicode61");')  # UNINDEXED

CPU times: user 5.15 ms, sys: 109 ms, total: 114 ms
Wall time: 146 ms


In [53]:
%%time
cur.executemany(
    "insert into imdb (text) values (?,?);",
    df[["text", "title"]].to_records(index=False),
)
db.commit()

CPU times: user 3min 57s, sys: 25.3 s, total: 4min 23s
Wall time: 4min 26s


In [70]:
res = cur.execute(
    f"""INSERT INTO imdb(imdb) VALUES('optimize');""",
)
db.commit()

In [11]:
import re

from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))  # 使用言語に応じて変更

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
file_path = "tutorial.db"
db = sqlite3.connect(file_path)
cur = db.cursor()

In [35]:
res = cur.execute(
    f"""CREATE VIRTUAL TABLE ft1_r USING fts5vocab('imdb', 'row');""",
)
db.commit()

In [57]:
%%time


index = 0
text = (
    df.iloc[index].prompt
    + " "
    + df.iloc[index].A
    + " "
    + df.iloc[index].B
    + " "
    + df.iloc[index].C
    + " "
    + df.iloc[index].D
    + " "
    + df.iloc[index].E
)
text = re.sub(r"[^a-zA-Z0-9]", " ", text)
filtered_words = [word for word in text.split() if word.lower() not in stop_words]


res = cur.execute(
    f"""SELECT * FROM ft1_v WHERE term IN ({','.join(['?']*len(filtered_words))}) AND cnt < 1000 ORDER BY cnt""",
    filtered_words,
).fetchall()

res

CPU times: user 181 ms, sys: 0 ns, total: 181 ms
Wall time: 179 ms


[('called', 'text', 1, 1),
 ('considered', 'text', 1, 1),
 ('mathematical', 'text', 4, 6),
 ('around', 'title', 285, 285)]

In [58]:
%%time

q = " OR ".join([row[0] for row in res])

res = cur.execute(
    f"""select text, rank
                      from imdb
                      where text MATCH "{q }"
                      ORDER BY rank
                      limit 5""",
).fetchall()

print(text)
print()
print(q)

res

Which of the following statements accurately describes the impact of Modified Newtonian Dynamics  MOND  on the observed  missing baryonic mass  discrepancy in galaxy clusters  MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called  fuzzy dark matter   MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20  MOND is a theory that explains the missing baryonic mass in galaxy clusters that was previously considered dark matter by demonstrating that the mass is in the form of neutrinos and axions  MOND is a theory that reduces the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 2  MOND is a theory that eliminates the observed missing baryonic mass in g

[("European Girls' Mathematical Olympiad > The European Girls' Mathematical Olympiad (EGMO) is a mathematical olympiad for girls which started in 2012, and is held in April each year. It was inspired by the China Girls Mathematical Olympiad (CGMO). Although the competition is held in Europe, it is open to female participants from all over the world, and is considered the most prestigious mathematics competition for girls. In recent years, participants from around 55 countries have been invited to the competition.",
  -17.294626741001327),
 ('Bhāskara II > Bhāskara and his works represent a significant contribution to mathematical and astronomical knowledge in the 12th century. He has been called the greatest mathematician of medieval India. His main work Siddhānta-Śiromaṇi, (Sanskrit for Crown of Treatises) is divided into four parts called Līlāvatī, Bījagaṇita, Grahagaṇita and Golādhyāya, which are also sometimes considered four independent works. These four sections deal with arithme

In [82]:
index = 6
df.iloc[index].prompt + df.iloc[index].A

'Which of the following statements accurately describes the dimension of an object in a CW complex?The dimension of an object in a CW complex is the largest n for which the n-skeleton is nontrivial, where the empty set is considered to have dimension -1 and the boundary of a discrete set of points is the empty set.'

In [84]:
df.iloc[index]

id                                                          6
prompt      Which of the following statements accurately d...
A           The dimension of an object in a CW complex is ...
B           The dimension of an object in a CW complex is ...
C           The dimension of an object in a CW complex is ...
D           The dimension of an object in a CW complex is ...
E           The dimension of an object in a CW complex dep...
answer                                                      A
context     Dimension > Moreover, the boundary of a discre...
sim_max                                              0.227938
sim_min                                               0.12335
sim_mean                                             0.203713
sim_std                                              0.027361
sim_num                                                    11
preds                                                       A
Name: 6, dtype: object